---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [6]:
def date_sorter():
    import pandas as pd
    import re
    
    doc = []
    with open('dates.txt') as file:
        for line in file:
            doc.append(line)

    df = pd.Series(doc)
    dates_df = pd.DataFrame(df)    
    
    # Creates a copy of text data
    dates_df = df.copy()
    # Finds YYYY anywhere in line
    dates_df.update(df[df.str.contains(r'\D(\d{4,4})\D')].str.findall(r'\D(\d{4,4})\D'))
    # Finds YYYY at the start of line
    dates_df.update(df[df.str.contains(r'^(\d{4,4})\D')].str.findall(r'^(\d{4,4})\D'))
    # Find str M (comma, space, or period) D (comma, space, or period) YY 
    dates_df.update(df[df.str.contains(r'(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\D{0,2}(\d{1,2})\D{1,4}(\d{4,4})')].str.findall(r'(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\D{0,2}(\d{1,2})\D{1,4}(\d{4,4})'))
    dates_df.update(df[df.str.contains(r'(January|Janaury|February|March|April|May|June|July|August|September|October|November|December|Decemeber)\D{0,2}(\d{1,2})\D{1,4}(\d{4,4})')].str.findall(r'(January|Janaury|February|March|April|May|June|July|August|September|October|November|December|Decemeber)\D{0,2}(\d{1,2})\D{1,4}(\d{4,4})'))
    # Finds D str M (comma, space, or period) YY
    dates_df.update(df[df.str.contains(r'(\d{2,2})\D([A-z][a-z]+)\D{1,2}(\d{4,4})')].str.findall(r'(\d{2,2})\D([A-z][a-z]+)\D{1,2}(\d{4,4})'))
    # Finds str MMM (comma, period) space YYYY
    dates_df.update(df[df.str.contains(r'(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[.,]?\s(\d{4,4})')].str.findall(r'(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[.,]?\s(\d{4,4})'))
    # Finds str MMM (comma, period) space YYYY
    dates_df.update(df[df.str.contains(r'(January|Janaury|February|March|April|May|June|July|August|September|October|November|December|Decemeber)[.,]?\s(\d{4,4})')].str.findall(r'(January|Janaury|February|March|April|May|June|July|August|September|October|November|December|Decemeber)[.,]?\s(\d{4,4})'))
    # Finds MM/YYYY or M/YYYY 
    dates_df.update(df[df.str.contains(r'(\d{1,2})/(\d{4,4})')].str.findall(r'(\d{1,2})/(\d{4,4})'))
    # Finds MM/DD/YYYY or M/D/YY anywhere
    dates_df.update(df[df.str.contains(r'(\d{1,2})/(\d{1,2})/(\d{2,4})')].str.findall(r'(\d{1,2})/(\d{1,2})/(\d{2,4})'))
    # Finds MM/DD/YYYY or M/D/YY anywhere
    dates_df.update(df[df.str.contains(r'\((\d{1,2})/(\d{1,2})/(\d{2,4})')].str.findall(r'(\d{1,2})/(\d{1,2})/(\d{2,4})\)'))
    # Finds MM-YYYY or M-YYYY 
    dates_df.update(df[df.str.contains(r'(\d{1,2})-(\d{4,4})')].str.findall(r'(\d{1,2})-(\d{4,4})'))
    # Finds MM-DD-YYYY or M-D-YY anywhere
    dates_df.update(df[df.str.contains(r'(\d{1,2})-(\d{1,2})-(\d{2,4})')].str.findall(r'(\d{1,2})-(\d{1,2})-(\d{2,4})'))
    # Finds str MMM-DD-YYYY
    dates_df.update(df[df.str.contains(r'(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\D{0,2}(\d{1,2})\D{1,4}(\d{4,4})')].str.findall(r'(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\D{0,2}(\d{1,2})\D{1,4}(\d{4,4})'))
    # Finds long str MMM-DD-YYYY
    dates_df.update(df[df.str.contains(r'(January|Janaury|February|March|April|May|June|July|August|September|October|November|December|Decemeber)\D{0,2}(\d{1,2})\D{1,4}(\d{4,4})')].str.findall(r'(January|Janaury|February|March|April|May|June|July|August|September|October|November|December|Decemeber)\D{0,2}(\d{1,2})\D{1,4}(\d{4,4})'))
    # Manual Corrections
    dates_df.iloc[271] = [('August', '2008')] 
    
    # Converts series to DF and creates date variales
    dates_df = pd.DataFrame(dates_df)
    dates_df.columns = ['extract']
    dates_df['month'] = 1
    dates_df['day'] = 1
    dates_df['year'] = 1
    
    # Creates dic for months to num
    months = {'Jan':1, 'Feb':2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, 'Jul':7, 'Aug':8, 'Sep':9, 'Oct':10, 'Nov':11, 'Dec':12,
         'January':1, 'Janaury':1, 'February':2, 'March':3, 'April':4, 'May':5, 'June':6, 'July':7, 'August':8, 'September':9, 
          'October':10, 'November':11, 'December':12, 'Decemeber':12}
    
    # Converts extracted values to int and stores them in proper fields
    for num in range(0, len(dates_df)):  
        if len(dates_df['extract'].iloc[num][0]) == 4:
            dummy_year = int(dates_df['extract'].iloc[num][0])
            if dummy_year < 20:
                dates_df['year'].iloc[num] = dummy_year + 2000
            elif dummy_year > 1000:
                dates_df['year'].iloc[num] = dummy_year
            else:            
                dates_df['year'].iloc[num] = dummy_year + 1900
        if len(dates_df['extract'].iloc[num][0]) == 3:
            dummy_year = int(dates_df['extract'].iloc[num][0][2])
            if dummy_year < 20:
                dates_df['year'].iloc[num] = dummy_year + 2000
            elif dummy_year > 1000:
                dates_df['year'].iloc[num] = dummy_year
            else:            
                dates_df['year'].iloc[num] = dummy_year + 1900
            if dates_df['extract'].iloc[num][0][0] in months.keys():
                dates_df['month'].iloc[num] = months[dates_df['extract'].iloc[num][0][0]]
            else:
                dates_df['month'].iloc[num] =int(dates_df['extract'].iloc[num][0][0])
            dates_df['day'].iloc[num] =int(dates_df['extract'].iloc[num][0][1])
        if len(dates_df['extract'].iloc[num][0]) == 2:
            dummy_year = int(dates_df['extract'].iloc[num][0][1])
            if dummy_year < 20:
                dates_df['year'].iloc[num] = dummy_year + 2000
            elif dummy_year > 1000:
                dates_df['year'].iloc[num] = dummy_year
            else:            
                dates_df['year'].iloc[num] = dummy_year + 1900
            if dates_df['extract'].iloc[num][0][0] in months.keys():
                dates_df['month'].iloc[num] = months[dates_df['extract'].iloc[num][0][0]]
            else:
                dates_df['month'].iloc[num] =int(dates_df['extract'].iloc[num][0][0])
    # Makes dates
    dates_df['date'] = pd.to_datetime(dates_df[['year', 'month', 'day']])
    
    return dates_df['date'].sort_values().index